In [1]:
import requests as req
import pandas as pd
from pydantic import BaseModel
from typing import Any

In [2]:
r = req.get('https://api.dedust.io/v2/pools')
pools = pd.DataFrame(r.json()) 

r0 = pools['assets'].apply(pd.Series)[0]
r1 = pools['assets'].apply(pd.Series)[1]
tokens = pd.concat([r0, r1])

In [3]:
class Token(BaseModel):
    native: bool
    address: str
    name: str | None  
    symbol: str | None

def process_token(token: dict[str, Any]) -> dict[str, str | bool]:
    name = ''
    symbol = ''
    native = token.get('type') == 'native'
    address = '0x' if native else token.get('address','address not found')

    metadata = token.get('metadata', {})
    if metadata:
        name = metadata.get('name')
        symbol = metadata.get('symbol')

    return Token(native=native, address=address, name=name, symbol=symbol).model_dump()

In [4]:
data = []
for token in tokens:
    data.append(process_token(token))
tokens = pd.DataFrame(data)

tokens = tokens.drop_duplicates(subset=['address'])
tokens = tokens[['address','name','symbol']].rename({'address':'id'}, axis=1)
pools = pools['assets'].apply(pd.Series)

In [5]:
pools[0] = pools[0].apply(process_token)
pools[1] = pools[1].apply(process_token)
for column in pools.columns:
    pools[column] = pools[column].apply(lambda x: x['address'])
pools.columns = ['source','target']

In [6]:
tokens.to_csv('tokens.csv', index=False, sep=';')
pools.to_csv('pools.csv', index=False, sep=';')